In [1]:
%matplotlib nbagg

import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage

In [2]:
def plot_profile(x, y, x_range=None, bins=40, color='blue'):
    df = pd.DataFrame(data=dict(x=x, y=y))
    if x_range is None:
        x_range = min(x), max(x)
    xlims = np.linspace(*x_range, num=(bins+1))
    xvals = (xlims[:-1] + xlims[1:])/2
    yvals = np.zeros(bins, dtype=float)
    yerrs = np.zeros(bins, dtype=float)
    for i, xmin, xmax in zip(range(bins), xlims[:-1], xlims[1:]):
        my_df = df.query("x >= {} and x < {}".format(xmin, xmax))
        yvals[i] = np.mean(my_df['y'])
        yerrs[i] = np.std(my_df['y'])/np.sqrt(len(my_df['y']))
    plt.errorbar(xvals, yvals, yerr=yerrs, fmt='.', color=color)

In [3]:
def plot_diff_vs_mag(df, title='', y_range=None, figsize=(9, 6), fontsize=12):
    mags = df['mag']
    diffs = df['diff']
    maxDiff = 12
    nStdev = 3.0

    ptSize = 25
    color = 'b'

    left, width = 0.12, 0.62
    bottom, height = 0.10, 0.85 # had to change height to make it save correctly
    left_h = left + width + 0.03
    bottom_h = bottom + width + 0.02
    rect_scatter = [left, bottom, width, height]
    rect_histy = [left_h, bottom, 0.20, height]

    deltaMin = 0.0

    fig = plt.figure(figsize=figsize)
    axScatter = plt.axes(rect_scatter)
    plot_profile(mags, diffs, color='black')
    if title:
        plt.title(title)
    axHisty = plt.axes(rect_histy)

    axScatter.scatter(mags, diffs, s=1.3*ptSize, marker="o", facecolors="b",
                      edgecolors=color, alpha=0.15)
    axScatter.axhline(0, ls='--', color="0.4")
    
    # Show bin in magnitudes
    mag_bin_size = 0.1  # mag
    mag_bins = np.arange(16, 21, mag_bin_size)
    bin_index = np.digitize(mags, mag_bins)
    

    axScatterY1 = axScatter.get_ylim()[0]
    axScatterY2 = axScatter.get_ylim()[1]
    nyDecimal = int(-1.0*np.around(np.log10(0.05*abs(axScatterY2 - (axScatterY1 - deltaMin))) - 0.5))
    yBinwidth = max(0.5/10**nyDecimal, np.around(0.02*abs(axScatterY2 - (axScatterY1 - deltaMin)), nyDecimal))

    yBins = np.arange((axScatterY1 - deltaMin) - 0.5*yBinwidth, axScatterY2 + 0.55*yBinwidth, yBinwidth)
    axHisty.hist(diffs, bins=yBins, color='b', alpha=0.6, orientation="horizontal", label='test')

    if y_range is None:
        y_range = -maxDiff, maxDiff
    axScatter.set_ylim(*y_range)
    axHisty.set_ylim(*y_range)
    #axHisty.set_xscale("log", nonposy="clip")
    axHisty.set_xscale("log")
    
    axScatter.set_xlabel('mag')
    axScatter.set_ylabel('Percentage difference star - model size: (sqrt(<xx+yy>/2)')
    
    mean = np.mean(diffs)
    std = np.std(diffs)
    std_clip = np.std( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    mean_clip = np.mean ( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    
    spc = 0.04
    plt.text(0.05, 0.95-0*spc, "Star N       = %s"  %len(diffs), ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-1*spc, "stddev      = %.4f" %std,        ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-2*spc, "std_clip     = %.4f"%std_clip,   ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-3*spc, "mean        = %.4f" %mean,       ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    plt.text(0.05, 0.95-4*spc, "mean_clip = %.4f"   %mean_clip,  ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')


In [84]:
def get_shape_mag_diff(butler, visits, detector):
    star_sizes = []
    model_sizes = []
    mags = []
    star_xxs = []
    star_yys = []
    star_xys = []
    psf_xxs = []
    psf_yys = []
    psf_xys = []
    star_ras = []
    star_decs = []
    detectors = []
    my_visits = []
    for visit in visits:
        data_id = dict(visit=visit, detector=detector)
        try:
            cat = butler.get('src', dataId=data_id)  
            calexp = butler.get('calexp', dataId=data_id)
            calib = calexp.getPhotoCalib()
        except dafPersist.NoResults as e:
            print(e)
            continue
        mask = (cat['calib_psf_used'] == 1)
        #mask &= (cat['base_PixelFlags_flag_interpolated'] == 0)
        mask &= (cat['base_PixelFlags_flag_saturatedCenter'] == 0)
        mask &= (cat['base_PsfFlux_instFlux'] > 0)
        cat = cat[mask]
        star_ras.extend([_ for _ in cat['coord_ra']])
        star_decs.extend([_ for _ in cat['coord_dec']])
        star_xxs.extend([_ for _ in cat['base_SdssShape_xx']])
        star_yys.extend([_ for _ in cat['base_SdssShape_yy']])
        star_xys.extend([_ for _ in cat['base_SdssShape_xy']])
        psf_xxs.extend([_ for _ in cat['base_SdssShape_psf_xx']])
        psf_yys.extend([_ for _ in cat['base_SdssShape_psf_yy']])
        psf_xys.extend([_ for _ in cat['base_SdssShape_psf_xy']])
        star_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_xx'] + cat['base_SdssShape_yy']))])
        model_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_psf_xx'] + cat['base_SdssShape_psf_yy']))])
        for flux in cat['base_PsfFlux_instFlux']:
            mags.append(calib.instFluxToMagnitude(flux))
        detectors.extend([detector for _ in range(len(cat))])
        my_visits.extend([visit for _ in range(len(cat))])

    diffs = [100*(a - b)/b for a,b in zip(star_sizes, model_sizes)]
    diffs = np.asarray(diffs)

    df = pd.DataFrame(data=dict(mag=mags, diff=diffs, detector=detectors, visit=my_visits,Ixx=star_xxs,Iyy=star_yys,Ixy=star_xys,RA=star_ras,DEC=star_decs,Ixx_psf=psf_xxs,Iyy_psf=psf_yys,Ixy_psf=psf_xys))
    return df

In [14]:
repo = '/global/cscratch1/sd/jchiang8/desc/BF_studies/output'
butler = dafPersist.Butler(os.path.join(repo, 'rerun/jchiang/no_bf_corr'))
butler_bf = dafPersist.Butler(os.path.join(repo, 'rerun/jchiang/bf_corr'))

In [15]:
!ls /global/cscratch1/sd/jchiang8/desc/BF_studies/output

CALIB  _mapper	calibrations  raw  ref_cats  registry.sqlite3  rerun


In [16]:
visits = [734193]
detectors = range(189)
#detectors = range(10)

In [17]:
detector = detectors[0]
data_id = dict(visit=visits[0], detector=detector)
cat = butler.get('src', dataId=data_id)

In [64]:
def plot_mag_diffs_vs_mag(visits, butler, diff_mag_file, title=''):
    if not os.path.isfile(diff_mag_file):
        dfs = []
        for detector in detectors:
            sys.stdout.write('{} '.format(detector))
            dfs.append(get_shape_mag_diff(butler, visits, detector))
        print()
        df = pd.concat(dfs)
        df.to_pickle(diff_mag_file)
    else:
        df = pd.read_pickle(diff_mag_file)

    plot_diff_vs_mag(df, title, y_range=(-2, 2))
    return df

In [85]:
diff_mag_file_no_bf_corr = 'diff_mags_v{}_no_bf_corr.pkl'.format(visits[0])
plot_mag_diffs_vs_mag(visits, butler, diff_mag_file_no_bf_corr,
                      title='visit {}, no B/F correction'.format(visits[0]))
plt.savefig('v{}_no_B-F_correction.png'.format(visits[0]));

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 


<IPython.core.display.Javascript object>

In [86]:
diff_mag_file_bf_corr = 'diff_mags_v{}_bf_corr.pkl'.format(visits[0])
plot_mag_diffs_vs_mag(visits, butler_bf, diff_mag_file_bf_corr,
                      title='visit {}, with B/F correction'.format(visits[0]))
plt.savefig('v{}_B-F_correction.png'.format(visits[0]))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 


<IPython.core.display.Javascript object>

In [11]:
diff_mag_with_bf_amp_kernel = 'diff_mag_data_with_bf_amp_kernel.pkl'
if not os.path.isfile(diff_mag_with_bf_amp_kernel):
    dfs = []
    for detector in detectors:
        sys.stdout.write('{} '.format(detector))
        dfs.append(get_shape_mag_diff(butler_bf_amp, visits, detector))
    print()
    df_bf = pd.concat(dfs)
    df_bf.to_pickle(diff_mag_with_bf_amp_kernel)
else:
    df_bf = pd.read_pickle(diff_mag_with_bf_amp_kernel)

plot_diff_vs_mag(df_bf, "Visit {}: with B/F amp-level kernel".format(visits[0]), y_range=(-2, 2))
plt.savefig('v204706_with_BF_amp_kernel.png');

0 

NameError: name 'butler_bf_amp' is not defined

In [ ]:
df_bf.head()

In [ ]:
butler_itl = dafPersist.Butler(os.path.join(repo, 'rerun/jchiang/with_itl_kernel'))

In [ ]:
diff_mag_file = 'diff_mag_data_with_itl_kernel.pkl'
if not os.path.isfile(diff_mag_file):
    dfs = []
    for detector in detectors:
        sys.stdout.write('{} '.format(detector))
        dfs.append(get_shape_mag_diff(butler_itl, visits, detector))
    print()
    df_bf = pd.concat(dfs)
    df_bf.to_pickle(diff_mag_file)
else:
    df_bf = pd.read_pickle(diff_mag_file)

plot_diff_vs_mag(df_bf, "Visit {}: with B/F ITL kernel".format(visits[0]), y_range=(-2, 2))
plt.savefig('v204706_with_BF_itl_kernel.png');

In [ ]:
import glob
def plot_mean_diffs(df, max_mag=17, fig=None, label=None):
    dets = sorted(list(set(df['detector'])))
    mean_diff = []
    sigma_diff = []
    mean_mag = []
    for det in dets:
        cut = '(mag < {}) & (detector=={})'.format(max_mag, det)
        my_df = df.query(cut)
        diffs = my_df['diff']
        mean_diff.append(np.mean(diffs))
        sigma_diff.append(np.std(diffs))
        mean_mag.append(np.mean(my_df['mag']))
    if fig is None:
        fig = plt.figure()
    plt.errorbar(dets, mean_diff, yerr=sigma_diff, fmt='.', label=label)
    return fig, mean_diff, sigma_diff, mean_mag

max_mag = 17.5
diff_mag_files = sorted(glob.glob('diff_mag*.pkl'))
dfs = []
fig = None
mean_diffs = dict()
sigma_diffs = dict()
mean_mags = dict()
for item in diff_mag_files:
    dfs.append(pd.read_pickle(item))
    fig, mds, sds, mags = plot_mean_diffs(dfs[-1], max_mag=max_mag, fig=fig, label=item)
    mean_diffs[item] = mds
    sigma_diffs[item] = sds
    mean_mags[item] = mags
plt.legend(fontsize='x-small')
plt.figure()
plt.errorbar(mean_diffs['diff_mag_data_no_bf_no_sat.pkl'], mean_diffs['diff_mag_data_with_bf_amp_kernel.pkl'],
            fmt='.', label='C04 kernel vs no B/F correction')
plt.errorbar(mean_diffs['diff_mag_data_with_itl_kernel.pkl'], mean_diffs['diff_mag_data_with_bf_amp_kernel.pkl'],
            fmt='.', label='C04 kernel vs itl kernel')
plt.legend(fontsize='x-small')
plt.figure()
plt.errorbar(mean_mags['diff_mag_data_no_bf_no_sat.pkl'], mean_diffs['diff_mag_data_with_bf_amp_kernel.pkl'], fmt='.')

In [ ]:
plt.figure()
for item in diff_mag_files:
    plt.hist(mean_diffs[item], label=item, bins=30, range=(-1, 1), histtype='step')
plt.legend(fontsize='x-small')